In [ ]:
"""
Author: Daniel Wu
Last Updated: August 25th, 2020
Purpose: Build the Mehra and Prescott (1985) Model 
         Using OOP techniques         
"""

import numpy as np
from numpy import linalg 

# 2-State Markov Process:
class Two_State:
    def __init__(self, stay):
        self.phi = np.array([[stay, 1 - stay], [1 - stay, stay]]) 
        
# 3-State Markov Process:         
class Three_State:
    def __init__(self, stay, eta):
        self.phi = np.array([[stay, 1 - stay - eta, eta], 
                             [1 - stay - eta, stay, eta], 
                             [0.5, 0.5, 0]])  
        

# Equilibrium Consumption-Based Asset Pricing Model
class MehraPrescott:
    def __init__(self, mu, delta, phi, dfactor, alpha, lev = 1):
        
        self.dfactor = dfactor   
        self.alpha = alpha         
        self.phi = phi
                      
        self.pi = linalg.matrix_power(self.phi, 50)[1,:]
        
        self.n = len(self.phi)        
        self.k = lev        
                
        if self.n == 2:
            self.lambda0 = np.array([1 + mu + delta, 1 + mu - delta])
        elif self.n == 3:
            self.lambda0 = np.array([1 + mu + delta, 1 + mu - delta, (1 + mu)/2])
        
        self.lambda_mtx = np.diag(self.lambda0)        
        self.A_mtx = dfactor * np.matmul(self.phi, linalg.matrix_power(self.lambda_mtx, self.k - alpha))        
        W_mtx = np.matmul(linalg.inv(np.identity(self.n) - self.A_mtx), self.A_mtx)                         
        self.w = np.matmul(W_mtx, np.ones((self.n, 1))) 
        
    def Get_StockReturn(self):
        Re = np.matmul((1/self.w), ((1+self.w).T * np.power(self.lambda0, self.k)))
        self.Re = Re        
        Re_phi = self.phi * Re
        ERe_cond = Re_phi.sum(axis = 1)                         
        ERe_uncond = np.matmul(ERe_cond.T, self.pi)
        
        Re_phi_sq = self.phi * np.power(Re, 2)        
        ERe_cond_sq = Re_phi_sq.sum(axis = 1) 
        ERe_uncond_sq = np.matmul(ERe_cond_sq.T, self.pi)                
        Var_Re = ERe_uncond_sq - np.power(ERe_uncond,2)
        
        self.ER_Stock = ERe_uncond - 1
        self.SD_Stock = np.power(Var_Re, .5)
        
    def Get_BondReturn(self):
        Rb_cond = 1 / (self.dfactor * np.matmul(self.phi, np.power(self.lambda0, -self.alpha)))
        self.Rb_cond = Rb_cond
        Rb_uncond = np.matmul(Rb_cond.T, self.pi)                
        Rb_uncond_sq = np.matmul(np.power(Rb_cond, 2).T, self.pi)        
        Var_Rb = Rb_uncond_sq - np.power(Rb_uncond, 2)
    
        self.Rf = Rb_uncond - 1
        self.SD_Rf = np.power(Var_Rb, .5)                
        
    def Get_ExcessReturn(self):                
        RExcess = self.Re - np.tile(self.Rb_cond, (self.n, 1)).T        
        
        RExcess_phi = self.phi * RExcess
        E_RExcess_cond = RExcess_phi.sum(axis = 1)
        E_RExcess_uncond = np.matmul(E_RExcess_cond.T, self.pi)
          
        RExcess_phi_sq = self.phi * np.power(RExcess, 2)
        E_RExcess_cond_sq = RExcess_phi_sq.sum(axis = 1)
        E_RExcess_uncond_sq = np.matmul(E_RExcess_cond_sq.T, self.pi)
        Var_RExcess = E_RExcess_uncond_sq - np.power(E_RExcess_uncond,2)
        
        self.ER_Excess = E_RExcess_uncond 
        self.SD = np.power(Var_RExcess, .5)        
        self.Sharpe = self.ER_Excess / self.SD